# Get tweets with given hashtag from Twitter

For a user-specified hashtag, grabs tweets from the Twitter API and dumps them once an hour into a data folder. Also unfolds the user and entities part of the returned data, to give you nice clean data columns. 

This code will run over an hour at a time, then you'll need to rerun the last cell to get more data. A better way to do this is to run the code from a cron job, as a .py file

In [2]:
# Set up the link to Twitter
import twitter
import pandas as pd
import time
import datetime
import re


fsecret = open('../../../../../../twittersecrets.txt', 'r')
secrets = fsecret.readline()
ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET = secrets.strip().split(',')
ACCESS_TOKEN = ACCESS_TOKEN.strip()
ACCESS_TOKEN_SECRET = ACCESS_TOKEN_SECRET.strip()
CONSUMER_KEY = CONSUMER_KEY.strip()
CONSUMER_SECRET = CONSUMER_SECRET.strip()

auth = twitter.oauth.OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
ta = twitter.Twitter(auth=auth)

In [14]:
# Get your first page of data
hashtag = '#qanon'
tweetjson = ta.search.tweets(q=hashtag)
dftweet = pd.DataFrame(tweetjson['statuses'])
dftweet = pd.concat([dftweet, dftweet['user'].apply(pd.Series).add_prefix('user_'),
                     dftweet['entities'].apply(pd.Series).add_prefix('entity_')], axis=1)
maxid = dftweet['id'].iloc[-1]-1 #ungodly hack because I haven't had enough coffee - google!
dftweet

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,user_time_zone,user_translator_type,user_url,user_utc_offset,user_verified,entity_hashtags,entity_media,entity_symbols,entity_urls,entity_user_mentions
0,None,None,Fri Mar 30 02:14:11 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [94...",NaN,0,False,None,979542270990012418,979542270990012418,...,None,none,None,NaN,False,"[{'text': 'QAnon', 'indices': [94, 100]}, {'te...",NaN,[],[],"[{'screen_name': '2runtherace', 'name': 'Runni..."
1,None,None,Fri Mar 30 02:14:10 +0000 2018,"{'hashtags': [{'text': 'FakeNews', 'indices': ...",NaN,0,False,None,979542266959286272,979542266959286272,...,None,none,https://t.co/N22froZigN,NaN,False,"[{'text': 'FakeNews', 'indices': [32, 41]}]",NaN,[],[],"[{'screen_name': 'sorensh40', 'name': 'sorensh..."
2,None,None,Fri Mar 30 02:14:10 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [69...","{'media': [{'id': 979541083855048704, 'id_str'...",0,False,None,979542266703433729,979542266703433729,...,Central Time (US & Canada),none,None,-18000.0,False,"[{'text': 'QAnon', 'indices': [69, 75]}, {'tex...","[{'id': 979541083855048704, 'id_str': '9795410...",[],[],"[{'screen_name': 'occulturalism', 'name': '[o]..."
3,None,None,Fri Mar 30 02:14:09 +0000 2018,"{'hashtags': [{'text': 'Qanon', 'indices': [22...",NaN,0,False,None,979542263905882112,979542263905882112,...,None,none,None,NaN,False,"[{'text': 'Qanon', 'indices': [22, 28]}]",NaN,[],[],"[{'screen_name': 'prayingmedic', 'name': 'Pray..."
4,None,None,Fri Mar 30 02:14:09 +0000 2018,"{'hashtags': [{'text': 'LockHerUp', 'indices':...",NaN,0,False,None,979542263176015872,979542263176015872,...,None,none,None,NaN,False,"[{'text': 'LockHerUp', 'indices': [60, 70]}, {...",NaN,[],[],"[{'screen_name': 'SoulOnJourney11', 'name': 'M..."
5,None,None,Fri Mar 30 02:14:07 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [29...","{'media': [{'id': 979541989975707653, 'id_str'...",0,False,None,979542257400508416,979542257400508416,...,America/Chicago,none,None,-18000.0,False,"[{'text': 'QAnon', 'indices': [29, 35]}]","[{'id': 979541989975707653, 'id_str': '9795419...",[],[],"[{'screen_name': 'cjtruth', 'name': 'CJTRUTH',..."
6,None,None,Fri Mar 30 02:14:07 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [41...",NaN,0,False,None,979542256137986049,979542256137986049,...,Eastern Time (US & Canada),none,http://t.co/YYYoGGhMu8,-14400.0,False,"[{'text': 'QAnon', 'indices': [41, 47]}]",NaN,[],[],"[{'screen_name': 'AARNO_XXII', 'name': 'AARNO ..."
7,None,None,Fri Mar 30 02:14:07 +0000 2018,"{'hashtags': [{'text': 'HRCVideo', 'indices': ...",NaN,0,False,None,979542255118766080,979542255118766080,...,None,none,None,NaN,False,"[{'text': 'HRCVideo', 'indices': [21, 30]}, {'...",NaN,[],[],"[{'screen_name': 'Catheri75157424', 'name': 'C..."
8,None,None,Fri Mar 30 02:14:05 +0000 2018,"{'hashtags': [{'text': 'Qanon', 'indices': [43...","{'media': [{'id': 979405205354590208, 'id_str'...",0,False,None,979542247283699712,979542247283699712,...,None,none,None,NaN,False,"[{'text': 'Qanon', 'indices': [43, 49]}]","[{'id': 979405205354590208, 'id_str': '9794052...",[],[],"[{'screen_name': 'prayingmedic', 'name': 'Pray..."
9,None,None,Fri Mar 30 02:14:05 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,979542246474182656,979542246474182656,...,None,none,None,NaN,False,[],NaN,[],[],"[{'screen_name': 'DestroyIllusion', 'name': 'J..."


In [33]:
# Now work backwards in time, grabbing more data each time
# This bit of code should take an hour to run... and then dump out a timestamped file.
for j in range(4):
    try:
        for i in range(180):
            print('{}: grabbing backwards from {}'.format(i, maxid))
            tweetjson = ta.search.tweets(q=hashtag, max_id=maxid)
            dftweet = dftweet.append(pd.DataFrame(tweetjson['statuses']))
            dftweet = pd.concat([dftweet, dftweet['user'].apply(pd.Series).add_prefix('user_'),
                                 dftweet['entities'].apply(pd.Series).add_prefix('entity_')], axis=1)
            maxid = dftweet['id'].iloc[-1]-1
            #time.sleep(1)
    except:
        print('Dammit, twitter timed us out')
        time.sleep(15*60)

#Send this batch of tweets to a CSV file
outfile = '../data/qanon_tweets_{}.csv'.format(re.sub('[^\d]+', '_', str(datetime.datetime.now())))
dftweet.to_csv(outfile, index=False)
print('{} rows sent to file {}'.format(len(dftweet), outfile))
print(dftweet.head())
dftweet = pd.DataFrame([])

0: grabbing backwards from 979542189322719231
Dammit, twitter timed us out
29 rows sent to file ../data/qanon_tweets_2018_03_29_19_39_21_259927.csv


,contributors,coordinates,created_at,entities,entity_hashtags,entity_media,entity_symbols,entity_urls,entity_user_mentions,extended_entities,...,user_time_zone,user_translator_type,user_url,user_utc_offset,user_verified,entity_hashtags,entity_media,entity_symbols,entity_urls,entity_user_mentions
0,None,None,Fri Mar 30 02:14:11 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [94...","[{'text': 'QAnon', 'indices': [94, 100]}, {'te...",NaN,[],[],"[{'screen_name': '2runtherace', 'name': 'Runni...",NaN,...,None,none,None,NaN,False,"[{'text': 'QAnon', 'indices': [94, 100]}, {'te...",NaN,[],[],"[{'screen_name': '2runtherace', 'name': 'Runni..."
1,None,None,Fri Mar 30 02:14:10 +0000 2018,"{'hashtags': [{'text': 'FakeNews', 'indices': ...","[{'text': 'FakeNews', 'indices': [32, 41]}]",NaN,[],[],"[{'screen_name': 'sorensh40', 'name': 'sorensh...",NaN,...,None,none,https://t.co/N22froZigN,NaN,False,"[{'text': 'FakeNews', 'indices': [32, 41]}]",NaN,[],[],"[{'screen_name': 'sorensh40', 'name': 'sorensh..."
2,None,None,Fri Mar 30 02:14:10 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [69...","[{'text': 'QAnon', 'indices': [69, 75]}, {'tex...","[{'id': 979541083855048704, 'id_str': '9795410...",[],[],"[{'screen_name': 'occulturalism', 'name': '[o]...","{'media': [{'id': 979541083855048704, 'id_str'...",...,Central Time (US & Canada),none,None,-18000.0,False,"[{'text': 'QAnon', 'indices': [69, 75]}, {'tex...","[{'id': 979541083855048704, 'id_str': '9795410...",[],[],"[{'screen_name': 'occulturalism', 'name': '[o]..."
3,None,None,Fri Mar 30 02:14:09 +0000 2018,"{'hashtags': [{'text': 'Qanon', 'indices': [22...","[{'text': 'Qanon', 'indices': [22, 28]}]",NaN,[],[],"[{'screen_name': 'prayingmedic', 'name': 'Pray...",NaN,...,None,none,None,NaN,False,"[{'text': 'Qanon', 'indices': [22, 28]}]",NaN,[],[],"[{'screen_name': 'prayingmedic', 'name': 'Pray..."
4,None,None,Fri Mar 30 02:14:09 +0000 2018,"{'hashtags': [{'text': 'LockHerUp', 'indices':...","[{'text': 'LockHerUp', 'indices': [60, 70]}, {...",NaN,[],[],"[{'screen_name': 'SoulOnJourney11', 'name': 'M...",NaN,...,None,none,None,NaN,False,"[{'text': 'LockHerUp', 'indices': [60, 70]}, {...",NaN,[],[],"[{'screen_name': 'SoulOnJourney11', 'name': 'M..."
5,None,None,Fri Mar 30 02:14:07 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [29...","[{'text': 'QAnon', 'indices': [29, 35]}]","[{'id': 979541989975707653, 'id_str': '9795419...",[],[],"[{'screen_name': 'cjtruth', 'name': 'CJTRUTH',...","{'media': [{'id': 979541989975707653, 'id_str'...",...,America/Chicago,none,None,-18000.0,False,"[{'text': 'QAnon', 'indices': [29, 35]}]","[{'id': 979541989975707653, 'id_str': '9795419...",[],[],"[{'screen_name': 'cjtruth', 'name': 'CJTRUTH',..."
6,None,None,Fri Mar 30 02:14:07 +0000 2018,"{'hashtags': [{'text': 'QAnon', 'indices': [41...","[{'text': 'QAnon', 'indices': [41, 47]}]",NaN,[],[],"[{'screen_name': 'AARNO_XXII', 'name': 'AARNO ...",NaN,...,Eastern Time (US & Canada),none,http://t.co/YYYoGGhMu8,-14400.0,False,"[{'text': 'QAnon', 'indices': [41, 47]}]",NaN,[],[],"[{'screen_name': 'AARNO_XXII', 'name': 'AARNO ..."
7,None,None,Fri Mar 30 02:14:07 +0000 2018,"{'hashtags': [{'text': 'HRCVideo', 'indices': ...","[{'text': 'HRCVideo', 'indices': [21, 30]}, {'...",NaN,[],[],"[{'screen_name': 'Catheri75157424', 'name': 'C...",NaN,...,None,none,None,NaN,False,"[{'text': 'HRCVideo', 'indices': [21, 30]}, {'...",NaN,[],[],"[{'screen_name': 'Catheri75157424', 'name': 'C..."
8,None,None,Fri Mar 30 02:14:05 +0000 2018,"{'hashtags': [{'text': 'Qanon', 'indices': [43...","[{'text': 'Qanon', 'indices': [43, 49]}]","[{'id': 979405205354590208, 'id_str': '9794052...",[],[],"[{'screen_name': 'prayingmedic', 'name': 'Pray...","{'media': [{'id': 979405205354590208, 'id_str'...",...,None,none,None,NaN,False,"[{'text': 'Qanon', 'indices': [43, 49]}]","[{'id': 979405205354590208, 'id_str': '9794052...",[],[],"[{'screen_name': 'prayingmedic', 'name': 'Pray..."
9,None,None,Fri Mar 30 02:14:05 +0000 2018,"